In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(500)

In [3]:
Corpus = pd.read_csv('Hope_ENG_train.csv',usecols=['Text','Label'])

Corpus

,Text,Label
0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech
1,@Champions Again He got killed for using false...,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Bla...,Non_hope_speech
4,Whenever we say black isn't that racists? Why...,Non_hope_speech
...,...,...
22735,It's a load of bollocks every life matters sim...,Non_hope_speech
22736,no say it because all lives matter! deku would...,Non_hope_speech
22737,God says her life matters,Non_hope_speech
22738,This video is just shit. A bunch of whiny ass ...,Non_hope_speech


In [4]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# Step - a : Remove blank rows if any.
Corpus['Text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['Text'] = [entry.lower() for entry in Corpus['Text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['Text']= [word_tokenize(entry) for entry in Corpus['Text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [7]:
Corpus

,Text,Label,text_final
0,"[these, tiktoks, radiate, gay, chaotic, energy...",Non_hope_speech,"['tiktoks', 'radiate', 'gay', 'chaotic', 'ener..."
1,"[@, champions, again, he, got, killed, for, us...",Non_hope_speech,"['champion', 'get', 'kill', 'use', 'false', 'm..."
2,"[it, 's, not, that, all, lives, do, n't, matter]",Non_hope_speech,"['life', 'matter']"
3,"[is, it, really, that, difficult, to, understa...",Non_hope_speech,"['really', 'difficult', 'understand', 'black',..."
4,"[whenever, we, say, black, is, n't, that, raci...",Non_hope_speech,"['whenever', 'say', 'black', 'racist', 'say', ..."
...,...,...,...
22735,"[it, 's, a, load, of, bollocks, every, life, m...",Non_hope_speech,"['load', 'bollock', 'every', 'life', 'matter',..."
22736,"[no, say, it, because, all, lives, matter, !, ...",Non_hope_speech,"['say', 'life', 'matter', 'deku', 'would', 'sa..."
22737,"[god, says, her, life, matters]",Non_hope_speech,"['god', 'say', 'life', 'matter']"
22738,"[this, video, is, just, shit, ., a, bunch, of,...",Non_hope_speech,"['video', 'shit', 'bunch', 'whiny', 'as', 'bit..."


UNIGRAMME

In [24]:
Corpus = pd.read_csv('unigrame.csv',usecols=['Text','Label'])
Corpus = Corpus.set_axis(['text_final','Label'], axis=1, inplace=False)
Corpus

,text_final,Label
0,"['I', 'lives', '', 'people', 'agree', 'love', ...",Hope_speech
1,"['I', 'lives', '', 'matter', 'people', 'black'...",Non_hope_speech


In [25]:
Corpus1 = pd.read_csv('Hope_ENG_dev.csv')
Corpus1.columns =['Text', 'Label']
Corpus1

,Text,Label
0,@Generation X Counting money that she been giv...,Non_hope_speech
1,@Paola Hernandez i never said to be intolerant...,Non_hope_speech
2,@Firstlast300 Wow An opinion is that I don't l...,Non_hope_speech
3,WOW!!!!!!!That was so so inspiring and incredi...,Hope_speech
4,@FALC0n Yea sorry I know Asian is an ethnicit...,Non_hope_speech
...,...,...
2835,Such fake sentiment. .,Non_hope_speech
2836,@A G black lives arent undervalued compared to...,Non_hope_speech
2837,People who pulled it down can and will be arre...,Non_hope_speech
2838,@Aaron Castellanos It will be a two hour movie...,Non_hope_speech


In [26]:
# Step - a : Remove blank rows if any.
Corpus1['Text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus1['Text'] = [entry.lower() for entry in Corpus1['Text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus1['Text']= [word_tokenize(entry) for entry in Corpus1['Text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus1['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus1.loc[index,'text_final'] = str(Final_words)

In [27]:
Corpus1

,Text,Label,text_final
0,"[@, generation, x, counting, money, that, she,...",Non_hope_speech,"['generation', 'x', 'count', 'money', 'give', ..."
1,"[@, paola, hernandez, i, never, said, to, be, ...",Non_hope_speech,"['paola', 'hernandez', 'never', 'say', 'intole..."
2,"[@, firstlast300, wow, an, opinion, is, that, ...",Non_hope_speech,"['wow', 'opinion', 'like', 'color']"
3,"[wow, !, !, !, !, !, !, !, that, was, so, so, ...",Hope_speech,"['wow', 'inspiring', 'incredible', 'speech', '..."
4,"[@, falc0n, yea, sorry, i, know, asian, is, an...",Non_hope_speech,"['yea', 'sorry', 'know', 'asian', 'ethnicity',..."
...,...,...,...
2835,"[such, fake, sentiment, ., .]",Non_hope_speech,"['fake', 'sentiment']"
2836,"[@, a, g, black, lives, arent, undervalued, co...",Non_hope_speech,"['g', 'black', 'live', 'arent', 'undervalue', ..."
2837,"[people, who, pulled, it, down, can, and, will...",Non_hope_speech,"['people', 'pull', 'arrest', 'cause', 'damage'..."
2838,"[@, aaron, castellanos, it, will, be, a, two, ...",Non_hope_speech,"['aaron', 'castellanos', 'two', 'hour', 'movie..."


In [29]:
Train_X = Corpus['text_final']
Test_X = Corpus1['text_final']
Train_Y = Corpus['Label']
Test_Y = Corpus1['Label']

In [31]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [32]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [33]:
print(Tfidf_vect.vocabulary_)

{'lives': 483, 'people': 608, 'agree': 27, 'love': 497, 'like': 474, 'matter': 518, 'all': 31, 'still': 763, 'think': 810, 'really': 666, 'one': 589, 'christian': 150, 'black': 101, 'life': 473, 'much': 555, 'men': 531, 'want': 875, 'my': 559, 'it': 426, 'and': 43, 'feel': 293, 'need': 563, 'make': 510, 'made': 505, 'the': 801, 'race': 652, 'this': 812, 'you': 929, 'thinking': 811, 'we': 884, 'but': 124, 'live': 482, 'way': 883, 'thank': 797, 'say': 696, 'engineering': 265, 'women': 908, 'friends': 316, 'never': 565, 'why': 901, 'nothing': 575, 'god': 341, 'fit': 303, 'school': 699, 'she': 720, 'things': 809, 'many': 514, 'boy': 113, 'said': 691, 'created': 198, 'going': 343, 'means': 527, 'us': 856, 'disagree': 235, 'let': 468, 'go': 340, 'get': 331, 'always': 37, 'gave': 323, 'that': 799, 'age': 24, 'choice': 146, 'looking': 490, 'gender': 325, 'parents': 601, 'know': 455, 'thankful': 798, 'find': 298, 'forever': 308, 'family': 290, 'career': 133, 'yourself': 933, 'matters': 519, 'be

In [ ]:
print(Train_X_Tfidf)

In [35]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  73.66197183098592


In [36]:
predictions_SVM

array([1, 0, 1, ..., 1, 1, 1])

In [37]:
Test_Y

array([1, 1, 1, ..., 1, 1, 0])

In [38]:
from sklearn.metrics import classification_report

print(classification_report(Test_Y, predictions_SVM, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.19      0.52      0.28       272
           1       0.94      0.76      0.84      2568

    accuracy                           0.74      2840
   macro avg       0.56      0.64      0.56      2840
weighted avg       0.87      0.74      0.79      2840



BIGRAME

In [40]:
Corpus = pd.read_csv('bigrame.csv',usecols=['text_final','Label'])
Corpus

,text_final,Label
0,"['lives matter', 'I agree', 'All lives', 'I lo...",Hope_speech
1,"['lives matter', 'black lives', 'lives matter....",Non_hope_speech


In [41]:
Corpus1 = pd.read_csv('Hope_ENG_dev.csv')
Corpus1.columns =['Text', 'Label']
Corpus1

,Text,Label
0,@Generation X Counting money that she been giv...,Non_hope_speech
1,@Paola Hernandez i never said to be intolerant...,Non_hope_speech
2,@Firstlast300 Wow An opinion is that I don't l...,Non_hope_speech
3,WOW!!!!!!!That was so so inspiring and incredi...,Hope_speech
4,@FALC0n Yea sorry I know Asian is an ethnicit...,Non_hope_speech
...,...,...
2835,Such fake sentiment. .,Non_hope_speech
2836,@A G black lives arent undervalued compared to...,Non_hope_speech
2837,People who pulled it down can and will be arre...,Non_hope_speech
2838,@Aaron Castellanos It will be a two hour movie...,Non_hope_speech


In [42]:
# Step - a : Remove blank rows if any.
Corpus1['Text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus1['Text'] = [entry.lower() for entry in Corpus1['Text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus1['Text']= [word_tokenize(entry) for entry in Corpus1['Text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus1['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus1.loc[index,'text_final'] = str(Final_words)

In [43]:
Corpus1

,Text,Label,text_final
0,"[@, generation, x, counting, money, that, she,...",Non_hope_speech,"['generation', 'x', 'count', 'money', 'give', ..."
1,"[@, paola, hernandez, i, never, said, to, be, ...",Non_hope_speech,"['paola', 'hernandez', 'never', 'say', 'intole..."
2,"[@, firstlast300, wow, an, opinion, is, that, ...",Non_hope_speech,"['wow', 'opinion', 'like', 'color']"
3,"[wow, !, !, !, !, !, !, !, that, was, so, so, ...",Hope_speech,"['wow', 'inspiring', 'incredible', 'speech', '..."
4,"[@, falc0n, yea, sorry, i, know, asian, is, an...",Non_hope_speech,"['yea', 'sorry', 'know', 'asian', 'ethnicity',..."
...,...,...,...
2835,"[such, fake, sentiment, ., .]",Non_hope_speech,"['fake', 'sentiment']"
2836,"[@, a, g, black, lives, arent, undervalued, co...",Non_hope_speech,"['g', 'black', 'live', 'arent', 'undervalue', ..."
2837,"[people, who, pulled, it, down, can, and, will...",Non_hope_speech,"['people', 'pull', 'arrest', 'cause', 'damage'..."
2838,"[@, aaron, castellanos, it, will, be, a, two, ...",Non_hope_speech,"['aaron', 'castellanos', 'two', 'hour', 'movie..."


In [44]:
Train_X = Corpus['text_final']
Test_X = Corpus1['text_final']
Train_Y = Corpus['Label']
Test_Y = Corpus1['Label']

In [45]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [46]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [47]:
print(Tfidf_vect.vocabulary_)

{'lives': 1662, 'matter': 1768, 'agree': 111, 'all': 133, 'love': 1697, 'want': 3041, 'feel': 1031, 'like': 1641, 'my': 1903, 'mom': 1858, 'high': 1286, 'school': 2480, 'never': 1942, 'grew': 1195, 'shall': 2538, 'lay': 1591, 'think': 2848, 'black': 347, 'can': 450, 'disagree': 809, 'say': 2473, 'gave': 1123, 'us': 2995, 'mature': 1773, 'age': 105, 'network': 1940, 'engineer': 919, 'here': 1275, '23': 23, 'currently': 696, 'working': 3136, 'instructor': 1439, 'teaching': 2804, 'men': 1795, 'women': 3127, 'looking': 1682, 'it': 1470, 'next': 1949, 'teach': 2802, 'university': 2981, 'still': 2709, 'hiding': 1284, 'gender': 1129, 'parents': 2096, 'know': 1556, 'dating': 720, 'someone': 2641, 'friends': 1097, 'knows': 1559, 'already': 145, 'supportive': 2761, 'and': 164, 'thankful': 2824, 'find': 1047, 'true': 2930, 'without': 3120, 'peace': 2119, 'forever': 1076, 'randomgirlwhosings0804': 2309, 'why': 3101, 'need': 1931, 'sexuality': 2534, 'christian': 529, 'really': 2331, 'weird': 3072, 

In [48]:
print(Train_X_Tfidf)

  (0, 3189)	0.03464017470435592
  (0, 3188)	0.01732008735217796
  (0, 3186)	0.01732008735217796
  (0, 3185)	0.03464017470435592
  (0, 3184)	0.06928034940871183
  (0, 3179)	0.02598013102826694
  (0, 3174)	0.01732008735217796
  (0, 3173)	0.01732008735217796
  (0, 3155)	0.024342778442871648
  (0, 3150)	0.01732008735217796
  (0, 3145)	0.03464017470435592
  (0, 3139)	0.01732008735217796
  (0, 3138)	0.03464017470435592
  (0, 3136)	0.01732008735217796
  (0, 3135)	0.01732008735217796
  (0, 3134)	0.024342778442871648
  (0, 3133)	0.01732008735217796
  (0, 3130)	0.01732008735217796
  (0, 3128)	0.024342778442871648
  (0, 3127)	0.06928034940871183
  (0, 3125)	0.05196026205653388
  (0, 3122)	0.024342778442871648
  (0, 3120)	0.01732008735217796
  (0, 3101)	0.0433002183804449
  (0, 3093)	0.01732008735217796
  :	:
  (1, 28)	0.008732122874212888
  (1, 27)	0.008732122874212888
  (1, 26)	0.004366061437106444
  (1, 25)	0.004366061437106444
  (1, 24)	0.004366061437106444
  (1, 22)	0.002183030718553222
  (1,

In [49]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  75.21126760563381


In [52]:
predictions_SVM

array([1, 1, 1, ..., 1, 1, 1])

In [51]:
Test_Y

array([1, 1, 1, ..., 1, 1, 0])

In [53]:
from sklearn.metrics import classification_report

print(classification_report(Test_Y, predictions_SVM, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.21      0.56      0.30       272
           1       0.94      0.77      0.85      2568

    accuracy                           0.75      2840
   macro avg       0.57      0.67      0.58      2840
weighted avg       0.87      0.75      0.80      2840

